# Symbolic Computations in Julia 

## Import Packages 

In [1]:
using Symbolics
using SymbolicUtils

## Section 1: Introduction 

### List of Symbolics command 

1. collect
1. eval 
1. expand 
1. parse 


## Section 2: Manipulating Harmonics

Here we show how to compute $f^3(x)$, where $f(x) = A \cos(x) + B \sin(x)$ and truncate higher harmonics. We rely on term-rewriting using [SymbolicUtils](https://symbolicutils.juliasymbolics.org/rewrite/).   

To do: 
1. derivative computation, i.e., replace $x$ by $\omega t$ and compute $df/dt$ and $d^2 f/dt^2$. 
1. extend to vectors;
1. grab terms in $\cos(\omega t)$ and $\sin(\omega t)$ and parse these terms to a non-line 

In [62]:
@syms x::Real A::Real B::Real

r1 = @rule cos(~x)^3 => 0.75 * cos(~x) + 0.25 * cos(3 * ~x)
r2 = @rule sin(~x)^3 => 0.75 * sin(~x) - 0.25 * sin(3 * ~x)
r3 = @rule cos(~x)^2 => 1 - sin(~x)^2
r4 = @rule sin(~x)^2 => 1 - cos(~x)^2
r5 = @rule sin(3*~x) => 0 
r6 = @rule cos(3*~x) => 0 

expr1 = simplify(expand((A*cos(x)+B*sin(x))^3), RuleSet([r1,r2,r3,r4]))
expr2 = simplify(expand(expr1), RuleSet([r1,r2,r3,r4]))
expr3 = simplify(expand(expr2), RuleSet([r5,r6]))
#expr4 = collect(expr3)

0.75(A^3)*cos(x) + 0.75(A^2)*B*sin(x) + 0.75A*(B^2)*cos(x) + 0.75(B^3)*sin(x)

In [29]:
expr1

(A^3)*(0.75cos(x) + 0.25cos(3x)) + (B^3)*(0.75sin(x) - 0.25sin(3x)) + 3(A^2)*B*sin(x)*(1 - (sin(x)^2)) + 3A*(B^2)*cos(x)*(1 - (cos(x)^2))

## Section 3: Adding Constant to Ansatz 

In [30]:
@syms x::Real A::Real B::Real C::Real 

r1 = @rule cos(~x)^3 => 0.75 * cos(~x) + 0.25 * cos(3 * ~x)
r2 = @rule sin(~x)^3 => 0.75 * sin(~x) - 0.25 * sin(3 * ~x)
r3 = @rule cos(~x)^2 => 1 - sin(~x)^2
r4 = @rule sin(~x)^2 => 1 - cos(~x)^2
r5 = @rule sin(3*~x) => 0 
r6 = @rule cos(3*~x) => 0 

expr1 = simplify(expand((C+A*cos(x)+B*sin(x))^3), RuleSet([r1,r2,r3,r4]))
expr2 = simplify(expand(expr1), RuleSet([r1,r2,r3,r4]))
expr3 = simplify(expand(expr2), RuleSet([r5,r6]))

C^3 + 0.75(A^3)*cos(x) + 0.75(A^2)*B*sin(x) + 0.75A*(B^2)*cos(x) + 3.0A*(C^2)*cos(x) + 0.75(B^3)*sin(x) + 3.0B*(C^2)*sin(x) + 3.0(A^2)*C*(cos(x)^2) + 6.0A*B*C*sin(x)*cos(x) + 3.0(B^2)*C*(sin(x)^2)

## Section 4: Non-Linear Damping: Quadratic Velocity (fails)  

Equation to solve 

$$ 
m \, \ddot{u}(t) + \gamma \, \dot{u}(t) + \gamma_2 \, [ \dot{u}(t) ]^2 + k \, u(t) = 
F_0 \sin(\omega_d \, t) \, . 
$$

Here we assume a single frequency anzats for the position of the form   

$$ u(t) = C + A \, \cos(\omega_d \, t) + B \, \sin(\omega_d \, t) \, .   $$

We thus have that the velocity is given by 

$$ \dot{u}(t) = - \omega_d \, A \, \sin(\omega_d \, t) + \omega_d \, B \, \cos(\omega_d \, t) \, . $$

We assume the damping to be proportinal to the square of the velocity and thus are interested in computing 

$$ [ \dot{u}(t) ]^2 = [ - \omega_d \, A \, \sin(\omega_d \, t) + \omega_d \, B \, \cos(\omega_d \, t) ]^2 $$ 

After truncating harmonics (see below) we obtain that 

$$ [ \dot{u}(t) ]^2 = 0.5 \, \omega_d^2 \, A^2 + 0.5 \, \omega_d^2 \, B^2 \ . $$

therefore that $m \, \ddot{u}(t) + \gamma \, \dot{u}(t) + \gamma_2 \, [ \dot{u}(t) ]^2 + k \, u(t) = (- m \, \omega_d^2 \, A + \gamma \, \omega_d \, B + k \, A) \, \cos(\omega_d \, t) + (- m \, \omega_d^2 \, B - \gamma \, \omega_d \, B + k \, B) \, \sin(\omega_d \, t) + \gamma_2 \, ( 0.5 \, \omega_d^2 \, A^2 + 0.5 \, \omega_d^2 \, B^2 ) + k \, C $. The algebraic equations for $A$, $B$ and $C$ to be solved then become (same equations for $A$ and $B$, additional equation for $C$)  

$$
m \, ( \omega_0^2 - \omega_d^2) \, A + \gamma \, \omega_d \, B = 0 \\
- \gamma \, \omega_d \, A + m \, (\omega_0^2 - \omega_d^2 ) \, B = F_0 \\
\gamma_2 ( 0.5 \, \omega_d^2 \, A^2 + 0.5 \, \omega_d^2 \, B^2) + k \, C = 0  \, .
$$

This implies (???) that $C$ can be computed after solving a linear system for $A$ and $B$.  

In [34]:
@syms x::Real A::Real B::Real C::Real w::Real t::Real 

r1 = @rule sin(~w * ~t)cos(~w * ~t) => 0.5*sin(2* ~w * ~t)
r2 = @rule cos(~w * ~t)^2 => 0.5 + 0.5*cos(2* ~w * ~t)
r3 = @rule sin(~w * ~t)^2 => 0.5 - 0.5*cos(2* ~w * ~t)
r4 = @rule sin(2* ~w * ~t) => 0 
r5 = @rule cos(2* ~w * ~t) => 0 

expr1 = simplify(expand((-w*A*sin(w*t)+w*B*cos(w*t))^2))
expr2 = simplify(expand(expr1), RuleSet([r1]))
expr3 = simplify(expand(expr2), RuleSet([r2,r3]))
expr4 = simplify(expand(expr3), RuleSet([r4,r5]))

0.5(A^2)*(w^2) + 0.5(B^2)*(w^2)

## Section 5: Non-Linear Damping: Position Squared times Velocity  

Equation to solve 

$$ 
m \, \ddot{u}(t) + \gamma \, \dot{u}(t) + \gamma_2 \, [ u(t) ]^2 \dot{u}(t) + k \, u(t) = 
F_0 \sin(\omega_d \, t) \, . 
$$

Here we assume a single frequency anzats for the position of the form   

$$ u(t) = C + A \, \cos(\omega_d \, t) + B \, \sin(\omega_d \, t) \, .   $$

We thus have that the velocity is given by 

$$ \dot{u}(t) = - \omega_d \, A \, \sin(\omega_d \, t) + \omega_d \, B \, \cos(\omega_d \, t) \, . $$

We assume the damping to be proportinal to the square of position times the velocity and thus are interested in computing 

$$ \gamma_2 \, [ u(t) ]^2 \dot{u}(t) = [ C + A \, \cos(\omega_d \, t) + B \, \sin(\omega_d \, t) ]^2 [ - \omega_d \, A \, \sin(\omega_d \, t) + \omega_d \, B \, \cos(\omega_d \, t) ] $$ 

After truncating harmonics (see below) we obtain that 

$$
\gamma_2 \, [ u(t) ]^2 \dot{u}(t) = \gamma_2 \, [ - \omega_d \, A^2 \, C + \omega_d \, B^2 \, C ] \\
+ \gamma_2 \, [ 0.75 \, \omega_d \, B^3 - 0.5 \, \omega_d \, A^2 \, B + \omega_d \, B \, C^2 ] \, \cos(\omega_d \, t) \\
+ \gamma_2 \, [ - 0.75 \, \omega_d \, A^3 + 0.5 \, \omega_d \, A \, B^2 - \omega_d \, A \, C^2  ] \, \sin(\omega_d \, t)
$$

We therefore have that $m \, \ddot{u}(t) + \gamma \, \dot{u}(t) + \gamma_2 \, [ u(t)]^2 \dot{u}(t) + k \, u(t) = (- m \, \omega_d^2 \, A + \gamma \, \omega_d \, B + k \, A) \, \cos(\omega_d \, t) + (- m \, \omega_d^2 \, B - \gamma \, \omega_d \, B + k \, B) \, \sin(\omega_d \, t) + \gamma_2 \, ( \ldots) + k \, C $. The algebraic equations for $A$, $B$ and $C$ to be solved then become 

$$
m \, ( \omega_0^2 - \omega_d^2) \, A + \gamma \, \omega_d \, B + 
   \gamma_2 \, [ 0.75 \, \omega_d \, B^3 - 0.5 \, \omega_d \, A^2 \, B + \omega_d \, B \, C^2 ] = 0 \\
- \gamma \, \omega_d \, A + m \, (\omega_0^2 - \omega_d^2 ) \, B + 
  \gamma_2 \, [ - 0.75 \, \omega_d \, A^3 + 0.5 \, \omega_d \, A \, B^2 - \omega_d \, A \, C^2  ] = F_0 \\
\gamma_2 [ - \omega_d \, A^2 \, C + \omega_d \, B^2 \, C ] + k \, C = 0  \, .
$$

In case that $\gamma_2 = 0$ (no non-linear damping), then according to last equation $C = 0$ (as $k \neq 0$), as expected.  

In [66]:
@syms x::Real A::Real B::Real C::Real w::Real t::Real 

r1 = @rule cos(~x)^3 => 0.75 * cos(~x) + 0.25 * cos(3 * ~x)
r2 = @rule sin(~x)^3 => 0.75 * sin(~x) - 0.25 * sin(3 * ~x)
r3 = @rule sin(~w * ~t)cos(~w * ~t) => 0.5*sin(2* ~w * ~t)
r4 = @rule cos(~w * ~t)^2 => 0.5 + 0.5*cos(2* ~w * ~t)
r5 = @rule sin(~w * ~t)^2 => 0.5 - 0.5*cos(2* ~w * ~t)
r6 = @rule sin(2* ~w * ~t) => 0 
r7 = @rule cos(2* ~w * ~t) => 0 
r8 = @rule sin(3* ~w * ~t) => 0 
r9 = @rule cos(3* ~w * ~t) => 0 

expr1 = simplify(expand((C+A*sin(w*t)+B*cos(w*t))^2*(-w*A*sin(w*t)+w*B*cos(w*t))))
expr2 = simplify(expand(expr1), RuleSet([r1,r2]))
expr3 = simplify(expand(expr2), RuleSet([r3,r4,r5]))
expr4 = simplify(expand(expr3), RuleSet([r6,r7,r8,r9]))

-(A^2)*C*w + (B^2)*C*w - 0.75(A^3)*w*sin(t*w) - 0.5(A^2)*B*w*cos(t*w) + 0.5A*(B^2)*w*sin(t*w) - A*(C^2)*w*sin(t*w) + 0.75(B^3)*w*cos(t*w) + B*(C^2)*w*cos(t*w)

## Section 6: Sandbox 

In [32]:
@syms x y z
@rule(+(~~xs) => ~~xs)(x+y+z) 

3-element view(::Vector{Any}, 1:3) with eltype Any:
 z
 y
 x

In [27]:
@syms ω::Real
@variables A[1:5]::Real B[1:5]::Real 
@variables t

D = Differential(t)
D2 = Differential(t)^2

expr1 = A*cos(ω*t)+B*sin(ω*t)
expr2 = collect(A*expand_derivatives(D2(cos(ω*t)))+B*expand_derivatives(D2(sin(ω*t))))

5-element Vector{Num}:
 -A[1]*cos(t*ω)*(ω^2) - B[1]*sin(t*ω)*(ω^2)
 -A[2]*cos(t*ω)*(ω^2) - B[2]*sin(t*ω)*(ω^2)
 -A[3]*cos(t*ω)*(ω^2) - B[3]*sin(t*ω)*(ω^2)
 -A[4]*cos(t*ω)*(ω^2) - B[4]*sin(t*ω)*(ω^2)
 -A[5]*cos(t*ω)*(ω^2) - B[5]*sin(t*ω)*(ω^2)

In [56]:
@syms x y
simplify(expand((x+y)^2+2*(x+y)^2))

3(x^2 + y^2) + 6x*y

In [59]:
r12 = @rule sin((~-)*(~x)) => - sin((~x))

sin(~(-) * ~x) => -(sin(~x))

In [60]:
expr = simplify(sin(-x), r12)

-sin(x)